In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kraft

In [3]:
na_pa = {}

na_se_li_ = {}

for pa in kraft.path.get_child("../raw/"):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na)

    na_pa[na] = pa

    se_li_ = {
        ".": [],
        "<": [],
        ">": [],
    }

    se_ = []

    with open(pa) as io:

        for li in io:

            li = li.strip()

            if li == "":

                continue

            if li[0] == "#":

                se = li[2]

                se_.append(se)

                continue

            if li[0] == "(":

                assert li[-2] == "."

            se_li_[se].append(li)

    assert all(se in se_ for se in se_li_.keys())

    na_se_li_[na] = se_li_

len(na_se_li_)

242

In [4]:
for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"]:

        assert kraft.string.untitle(li) not in na_se_li_, "{}: {}".format(na, li)

In [5]:
import os
import shutil

In [6]:
for se_li_ in na_se_li_.values():

    se_li_["md<"] = []

    se_li_[">md"] = []

In [7]:
for na, se_li_ in na_se_li_.items():

    for li in list(se_li_[">"]):

        limi = kraft.string.untitle(li)

        if limi in na_se_li_:

            se_li_[">"].remove(li)

            se_li_[">md"].append(limi)

            na_se_li_[limi]["md<"].append(na)

In [8]:
import pandas as pd

In [9]:
na_na2 = pd.read_csv("../rename.tsv", "\t", index_col=0, squeeze=True).to_dict()

In [10]:
de = "\n\n"

for na, se_li_ in na_se_li_.items():

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            ".": [],
            "<": [],
            ">": [],
        }

        for li in se_li_["."]:

            ti_li_["."].append(li)

        for li in se_li_["<"]:

            ti_li_["<"].append(na_na2.get(li, li))

        for li in se_li_[">"] + se_li_[">md"]:

            ti_li_[">"].append(na_na2.get(li, li))

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            io.write(de.join(li_) + de)

SyntaxError: invalid syntax (<ipython-input-10-f032007e578f>, line 19)

In [ ]:
fe_ = []

for na, se_li_ in na_se_li_.items():

    fe_ += se_li_["<"]

    fe_ += se_li_[">"]

pd.DataFrame(index=sorted(set(fe_)), columns=["Phase"]).to_csv("../feature.tsv", "\t")

In [ ]:
di = "../compiled/"

if os.path.isdir(di):

    shutil.rmtree(di)

de = "\n\n"

for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            "Comment": [],
            "Epidemiology": [],
            "Cause": [],
            "Symptom": [],
            "Sign": [],
            "Diagnostic": [],
            "Treatment and Management": [],
        }

        for li in se_li_["."]:

            ti_li_["Comment"].append(li)

        for li in se_li_["<"]:

            ti_li_["Epidemiology"].append(li)

        for li in se_li_["md<"]:

            ti_li_["Cause"].append(li)

        for li in se_li_[">"] + kraft.iterable.flatten(
            [na_se_li_[li][">"] for li in se_li_[">md"]]
        ):

            if 1:

                ti_li_["Symptom"].append(li)

            elif 2:

                ti_li_["Sign"].append(li)

            elif 3:

                ti_li_["Diagnostic"].append(li)

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            for li, n in pd.value_counts(li_).items():

                if 1 < n:

                    so_ = [na]

                    so_ += [
                        liri for liri in se_li_[">md"] if li in na_se_li_[liri][">"]
                    ]

                    print("{}:\n\t{}\n".format(li, "\n\t".join(so_)))

            io.write(de.join(li_) + de)